<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/Forecasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Invoice Payment Forecasting

Accounts Receivables (AR) is an important aspect of businesses, it is a credit extended by the company to the client that require payments due within a time period. With B2B transactions increasing in volume and complexity, poor management of AR can lead to to unnecessary cash flow problems.

Invoice payment forecasting can be used to optimize collection strategies that allows businesses to predict and plan for future customer payments. By analyzing customer payment history, businesses can forecast their future accounts receivable balance and plan accordingly. By identifying invoices that are likely to be paid late, customers who are likely to have large amounts of overdue receivables, which in turn can lead to better visibility into future cash flow.

The invoices dataset contains the past payment information and behaviour of various buyers.

In [ ]:
%%bash
pip install --no-cache-dir -qU \
    datasets bitsandbytes peft
pip check

No broken requirements found.


In [2]:
import os

input_dir = './Data/Input'
os.makedirs(input_dir, exist_ok=True)

In [ ]:
%%bash
wget https://raw.githubusercontent.com/SkywalkerHub/Payment-Date-Prediction/main/Dataset.csv

In [6]:
import os
import pandas as pd

input_dir = './Data/Input'
input_payment = 'Dataset.csv'
df_payment = pd.read_csv(os.path.join(input_dir, input_payment))
df_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_code           50000 non-null  object 
 1   cust_number             50000 non-null  object 
 2   name_customer           50000 non-null  object 
 3   clear_date              45181 non-null  object 
 4   buisness_year           50000 non-null  float64
 5   doc_id                  50000 non-null  float64
 6   posting_date            50000 non-null  object 
 7   document_create_date    50000 non-null  int64  
 8   document_create_date.1  50000 non-null  int64  
 9   due_in_date             50000 non-null  float64
 10  invoice_currency        50000 non-null  object 
 11  document type           50000 non-null  object 
 12  posting_id              50000 non-null  float64
 13  area_business           0 non-null      float64
 14  total_open_amount       50000 non-null

In [7]:
df_payment

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,0200315290,KWI in,2019-03-07 00:00:00,2019.0,1.928827e+09,2019-02-20,20190220,20190220,20190307.0,USD,RV,1.0,NaN,21227.05,20190220.0,NAA8,1.928827e+09,0
1,U001,0200769623,WAL-MAR foundation,2020-01-16 00:00:00,2020.0,1.930350e+09,2020-01-04,20200103,20200104,20200119.0,USD,RV,1.0,NaN,5669.72,20200104.0,NAH4,1.930350e+09,0
2,U001,0200820380,SMITH'S associates,2019-07-08 00:00:00,2019.0,1.929517e+09,2019-06-21,20190619,20190621,20190706.0,USD,RV,1.0,NaN,67968.48,20190621.0,NAA8,1.929517e+09,0
3,U001,0200769623,WAL-MAR corp,2019-03-22 00:00:00,2019.0,1.928943e+09,2019-03-12,20190311,20190312,20190327.0,USD,RV,1.0,NaN,61395.73,20190312.0,NAH4,1.928943e+09,0
4,U001,200230690,DECA in,2019-06-03 00:00:00,2019.0,1.929350e+09,2019-05-21,20190521,20190521,20190608.0,USD,RV,1.0,NaN,1067.72,20190516.0,NAM4,1.929350e+09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U001,0200726979,BJ'S us,NaN,2020.0,1.930709e+09,2020-03-27,20200326,20200327,20200411.0,USD,RV,1.0,NaN,261.84,20200327.0,NAA8,1.930709e+09,1
49996,U001,CCCA02,KRAFT in,NaN,2020.0,1.930694e+09,2020-03-25,20200324,20200325,20200429.0,USD,RV,1.0,NaN,14151.51,20200325.0,NAG2,1.930694e+09,1
49997,U001,0200762301,C&S WH corporation,NaN,2020.0,1.930606e+09,2020-03-06,20200305,20200306,20200321.0,USD,RV,1.0,NaN,14898.82,20200306.0,NAC6,1.930606e+09,1
49998,U001,0100001196,DOLLAR,NaN,2020.0,1.930761e+09,2020-04-09,20200408,20200409,20200424.0,USD,RV,1.0,NaN,5127.95,20200409.0,NAA8,1.930761e+09,1


In [ ]:
# Import libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Reproducibility
seed = 42
set_seed(seed)

/Work/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-13 12:15:11.697825: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-13 12:15:11.697885: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-13 12:15:11.697943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Preprocessing

List of the fields in dataset are as follows:
* Customer: Customer code of the account
* Country: customer country
* Region:
* Account Owner Name
* Last Payment Date
* Contact Date
* Collection Specialist
* Projected Overdue Amount
* Current Overdue Amount
* Collected
* Forecast
* Forecast Status

In [ ]:
# Load the large language model file
from llama_cpp import Llama
LLM = Llama(model_path="./llama-2-7b-chat.ggmlv3.q8_0.bin")

prompt = "Q: Why are Jupyter notebooks difficult to maintain? A:"

output = LLM(prompt, max_tokens=0)

print(output["choices"][0]["text"])

In [ ]:
import xgboost as xgb